In [ ]:
import metapy
import pandas as pd
import numpy as np
from pyspark import SparkContext
import boto3

In [ ]:
# #s3 = boto3.client('s3')
# s3_session = boto3.Session(
#     aws_access_key_id="", 
#     aws_secret_access_key=""
# )

In [ ]:
# import sys
# sys.path.append('../utils/')

In [ ]:
# s3 = s3_session.resource('s3')
# s3_client = boto3.client('s3')
# # obj = s3.get_object(Bucket="st1800datasets", Key="datasets/news.csv")

In [ ]:
#response = s3_client.get_object(Bucket="st1800datasets",Key="datasets/news.csv")

In [ ]:
# We'll use a reduced version of the dataset while we configure our blob storage. The exercise does continue, albeit with fewer data to work on.
data = pd.read_csv('../datasets/news_reduced.csv', encoding="utf-8")

In [ ]:
data.head(20)

In [ ]:
news = data['title'] + ' ' + data['content']
metadata = data[['id_news','publication','author','date','year', 'month']]
metadata.head(10)

In [ ]:
!wget -N https://raw.githubusercontent.com/meta-toolkit/meta/master/data/lemur-stopwords.txt

In [ ]:
doc = metapy.index.Document()
doc.content(np.array2string(news.values, separator=' '))

In [ ]:
tok = metapy.analyzers.ICUTokenizer(suppress_tags=True)
tok = metapy.analyzers.LengthFilter(tok, min=2, max=40)
tok = metapy.analyzers.LowercaseFilter(tok)
tok.set_content(doc.content())
[t for t in tok]

In [ ]:
ana = metapy.analyzers.NGramWordAnalyzer(1, tok)